# Comparison of BirdNET, Perch and Finetuned model performance on Kenyan Species
This notebook is used to generate tables comparing the performance on BirdNET, Perch and a multi-label model trained using Perch embeddings to classify species from the Mt Kenya ecosystem.

In [1]:
import os
import json
import random
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from multilabel_mlp import *

from keras.utils import to_categorical, set_random_seed


2025-01-04 14:24:45.211796: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-04 14:24:45.211852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-04 14:24:45.213540: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-04 14:24:45.222435: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-04 14:24:46.106407: W tensorflow/compiler/tf2

In [2]:
def species_from_df(df):
    '''
    This function extracts species from the audio annotation files
    '''
    sps = []
    for i in range(len(df)):
        if not pd.isna(df.iloc[i]['Foreground Species']):
            sps += df.iloc[i]['Foreground Species'].split(';')
        if not pd.isna(df.iloc[i]['Background Species']):
            sps += df.iloc[i]['Background Species'].split(';')
    return list(set(sps))

In [3]:
annotations_dir = '../annotations/'

In [4]:
annotation_files = os.listdir(annotations_dir)
    
# audio 
aru_dfs = []
for filename in annotation_files:
    if 'aru' in filename  and filename.endswith('csv'):
    
        df = pd.read_csv(os.path.join(annotations_dir, filename))
        aru_dfs.append(df)
    
annotations = pd.concat(aru_dfs)
aru_cns = species_from_df(annotations)
aru_cns.sort()

# scientific names
df_sp = pd.read_csv('../annotations/Kenya-Species-List.csv')
aru_sns = [df_sp[df_sp['Common Name']==cn].iloc[0]['Scientific Name'] for cn in aru_cns]

In [5]:
# get number of files with species
num_files = 0
for indx, filename in enumerate(list(annotations['Filename'])):

    recording_cn = []
    if not pd.isna(annotations.iloc[indx]['Foreground Species']):
        recording_cn += annotations.iloc[indx]['Foreground Species'].split(';')
    if not pd.isna(annotations.iloc[indx]['Background Species']):
        recording_cn += annotations.iloc[indx]['Background Species'].split(';')
    
    if len(recording_cn):
        num_files += 1

recording_species = np.zeros((num_files, len(aru_sns)))

In [6]:
curr_file = 0    
recording_filename = []
for indx, filename in enumerate(list(annotations['Filename'])):

    recording_cn = []
    if not pd.isna(annotations.iloc[indx]['Foreground Species']):
        recording_cn += annotations.iloc[indx]['Foreground Species'].split(';')
    if not pd.isna(annotations.iloc[indx]['Background Species']):
        recording_cn += annotations.iloc[indx]['Background Species'].split(';')

    recording_sn = [df_sp[df_sp['Common Name']==cn].iloc[0]['Scientific Name'] for cn in recording_cn]

    if len(recording_cn):
        recording_filename.append(filename)
        
        for sp in recording_sn:
            recording_species[curr_file, aru_sns.index(sp)] = 1
            
        curr_file += 1
            

In [7]:
num_sp_rec = np.sum(recording_species, 0)

In [8]:
filtered_species = []
recording_threshold = 20

for i, status in enumerate(num_sp_rec >= recording_threshold):
    if status:
        filtered_species.append(aru_cns[i])
        

In [9]:
for sp in filtered_species:
    print(sp, num_sp_rec[aru_cns.index(sp)])

Black-backed Puffback 33.0
Brown Woodland Warbler 519.0
Cape Robin Chat 22.0
Chestnut-throated Apalis 132.0
Chin-spot Batis 38.0
Cinnamon Bracken Warbler 82.0
Cinnamon-chested Bee-eater 28.0
Collared Sunbird 41.0
Common Bulbul 150.0
Grey Apalis 28.0
Grey-backed Camaroptera 179.0
Hartlaub's Turaco 113.0
Montane White-eye 76.0
Mountain Yellow Warbler 77.0
Olive Thrush 31.0
Red-fronted Parrot 100.0
Rüppell's Robin Chat 29.0
Tambourine Dove 50.0
Tropical Boubou 113.0
Variable Sunbird 38.0
White-starred Robin 70.0
Yellow-breasted Apalis 22.0
Yellow-rumped Tinkerbird 124.0
Yellow-whiskered Greenbul 221.0


In [10]:
y = recording_species[:,num_sp_rec >= recording_threshold]

In [11]:
len(recording_filename)

1188

## Generate Table 9

In [12]:
# birdnet results obtained using birdnet_eval.py
birdnet_gt = np.load('aru_gt.npy')
birdnet_res = np.load('aru_pred.npy')


# perch results obtained using bvc_eval.py
perch_gt = np.load('aru_gt_bvc.npy')
perch_res = np.load('aru_pred_bvc.npy')

In [13]:
num_rec = []

pre_birdnet = []
recall_birdnet = []
f1_score_birdnet = []

pre_perch = []
recall_perch = []
f1_score_perch = []


for sp in filtered_species:
    indx = aru_cns.index(sp)
    num_rec.append(int(num_sp_rec[indx])),
    pre_birdnet.append(precision_score(birdnet_gt[:,indx], birdnet_res[:,indx]))
    recall_birdnet.append(recall_score(birdnet_gt[:,indx], birdnet_res[:,indx]))
    f1_score_birdnet.append(f1_score(birdnet_gt[:,indx], birdnet_res[:,indx]))
    
    pre_perch.append(precision_score(perch_gt[:,indx], perch_res[:,indx]))
    recall_perch.append(recall_score(perch_gt[:,indx], perch_res[:,indx]))
    f1_score_perch.append(f1_score(perch_gt[:,indx], perch_res[:,indx]))

/home/ftei-dsw/bird-recog/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ftei-dsw/bird-recog/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ftei-dsw/bird-recog/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ftei-dsw/bird-recog/li

In [14]:
# finetuned model performance generated by mlp-multi-species.ipynb
mlp_res = pd.read_csv('multi-species-mlp.csv')

In [15]:
mlp_f1 = []
for sp in filtered_species:
    mlp_f1.append(mlp_res[mlp_res['Common Name']==sp]['F1-Score'].iloc[0])

In [16]:
res_df = pd.DataFrame(zip(filtered_species, 
                          num_rec, 
                          pre_birdnet, 
                          recall_birdnet, 
                          f1_score_birdnet,
                          pre_perch, 
                          recall_perch, 
                          f1_score_perch,
                         mlp_f1),
                      columns=['Common Name', 'Number of Recordings',  'BirdNET Precision', 'BirdNET Recall', 'BirdNET F1-Score', 'Perch Precision', 'Perch Recall', 'Perch F1-Score', 'MLP F1-Score'])

In [17]:
res_df_ordered = res_df.sort_values(by=['Number of Recordings'], ascending=False)

In [18]:
print(res_df_ordered.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
)) 

\begin{tabular}{lrrrrrrrr}
\toprule
Common Name & Number of Recordings & BirdNET Precision & BirdNET Recall & BirdNET F1-Score & Perch Precision & Perch Recall & Perch F1-Score & MLP F1-Score \\
\midrule
Brown Woodland Warbler & 519 & 1.00 & 0.01 & 0.02 & 1.00 & 0.12 & 0.22 & 0.86 \\
Yellow-whiskered Greenbul & 221 & 0.93 & 0.40 & 0.56 & 0.94 & 0.53 & 0.68 & 0.69 \\
Grey-backed Camaroptera & 179 & 1.00 & 0.14 & 0.25 & 1.00 & 0.01 & 0.02 & 0.66 \\
Common Bulbul & 150 & 0.78 & 0.19 & 0.30 & 0.88 & 0.05 & 0.09 & 0.67 \\
Chestnut-throated Apalis & 132 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.42 \\
Yellow-rumped Tinkerbird & 124 & 1.00 & 0.27 & 0.43 & 0.95 & 0.33 & 0.49 & 0.58 \\
Hartlaub's Turaco & 113 & 0.93 & 0.34 & 0.49 & 0.94 & 0.29 & 0.45 & 0.62 \\
Tropical Boubou & 113 & 1.00 & 0.22 & 0.36 & 1.00 & 0.19 & 0.33 & 0.49 \\
Red-fronted Parrot & 100 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.53 \\
Cinnamon Bracken Warbler & 82 & 0.51 & 0.22 & 0.31 & 0.56 & 0.40 & 0.47 & 0.48 \\
Mo

In [19]:
print(res_df_ordered[['Common Name', 'Number of Recordings','BirdNET F1-Score','Perch F1-Score','MLP F1-Score']].to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
))

\begin{tabular}{lrrrr}
\toprule
Common Name & Number of Recordings & BirdNET F1-Score & Perch F1-Score & MLP F1-Score \\
\midrule
Brown Woodland Warbler & 519 & 0.02 & 0.22 & 0.86 \\
Yellow-whiskered Greenbul & 221 & 0.56 & 0.68 & 0.69 \\
Grey-backed Camaroptera & 179 & 0.25 & 0.02 & 0.66 \\
Common Bulbul & 150 & 0.30 & 0.09 & 0.67 \\
Chestnut-throated Apalis & 132 & 0.00 & 0.00 & 0.42 \\
Yellow-rumped Tinkerbird & 124 & 0.43 & 0.49 & 0.58 \\
Hartlaub's Turaco & 113 & 0.49 & 0.45 & 0.62 \\
Tropical Boubou & 113 & 0.36 & 0.33 & 0.49 \\
Red-fronted Parrot & 100 & 0.00 & 0.00 & 0.53 \\
Cinnamon Bracken Warbler & 82 & 0.31 & 0.47 & 0.48 \\
Mountain Yellow Warbler & 77 & 0.00 & 0.00 & 0.53 \\
Montane White-eye & 76 & 0.00 & 0.00 & 0.23 \\
White-starred Robin & 70 & 0.03 & 0.00 & 0.41 \\
Tambourine Dove & 50 & 0.30 & 0.29 & 0.55 \\
Collared Sunbird & 41 & 0.13 & 0.20 & 0.27 \\
Chin-spot Batis & 38 & 0.19 & 0.14 & 0.54 \\
Variable Sunbird & 38 & 0.05 & 0.05 & 0.25 \\
Black-backed Puffback & 3

In [20]:
res_df_ordered[['Common Name', 'Number of Recordings','BirdNET F1-Score','Perch F1-Score','MLP F1-Score']]

,Common Name,Number of Recordings,BirdNET F1-Score,Perch F1-Score,MLP F1-Score
1,Brown Woodland Warbler,519,0.019084,0.216495,0.862500
23,Yellow-whiskered Greenbul,221,0.561514,0.680115,0.692913
10,Grey-backed Camaroptera,179,0.245098,0.022099,0.660194
8,Common Bulbul,150,0.301075,0.088608,0.673267
3,Chestnut-throated Apalis,132,0.000000,0.000000,0.421053
22,Yellow-rumped Tinkerbird,124,0.430380,0.491018,0.580645
11,Hartlaub's Turaco,113,0.493506,0.445946,0.620690
18,Tropical Boubou,113,0.362319,0.325926,0.488372
15,Red-fronted Parrot,100,0.000000,0.000000,0.533333
5,Cinnamon Bracken Warbler,82,0.307692,0.468085,0.480000
